In [1]:
%matplotlib inline
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision.utils as vutils
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
import numpy as np
import matplotlib.pyplot as plt
import pickle
from metric import *
from utils import * 
import eval_iou_voc as ev
from fcn8 import FCN8_VGG16

In [2]:
dataroot = '/data/voc2012'
SAVE_DIR = '/data/model'

In [3]:
def to_tensor_target(img):
  img = np.array(img)
  # border
  img[img==255] = 0 # border = background 
  return torch.LongTensor(img)


size = (513,513)
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
transform_input = transforms.Compose([
                                   transforms.Resize(size),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                    ])

transform_mask = transforms.Compose([
                                   transforms.Resize(size),
                                   transforms.Lambda(to_tensor_target)
                                   ])

In [4]:
train_dataset = dset.VOCSegmentation(dataroot,year='2012', image_set='train', download=True,
                                     transform= transform_input,
                                     target_transform= transform_mask)

val_dataset = dset.VOCSegmentation(dataroot,year='2012', image_set='val', download=True,
                                     transform= transform_input,
                                     target_transform= transform_mask)

Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar


In [5]:
batch_size = 1

dataloader_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
dataloader_val = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
# Decide which device we want to run on
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device :",device)

device : cuda:0


## Load model

In [6]:
! ls /data/model

SVM_transfer_30_voc.pt		     deeplabv3_unsupervised_voc.pt  fcn16_bn.pt
cnn_classif_transfer_30_voc.pt	     deeplabv3_voc.pt		    fcn32.pt
deeplabv3_10_autosup_voc.pt	     dlv3_voc_sbd_100.pt	    fcn32_bn.pt
deeplabv3_30_semisup_g10_l2_voc.pt   dlv3_voc_sbd_30.pt		    fcn8.pt
deeplabv3_30_semisup_g50_alt_voc.pt  dlv3_voc_sbd_rotate_30.pt	    fcn8_bn.pt
deeplabv3_30_semisup_voc.pt	     dlv3_voc_sbd_semisup_g0.pt     iou
deeplabv3_30_voc.pt		     dlv3_voc_sbd_semisup_g05.pt    loss
deeplabv3_50_voc.pt		     dlv3_voc_sbd_semisup_g1.pt
deeplabv3_autosupervised_voc07.pt    fcn16.pt


In [7]:
def load_model(file=None):
    if file is None:
        model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=True)
        model.to(device)
    else:
        model = torch.load(os.path.join(SAVE_DIR,file))
    return model

In [8]:
model = load_model()
model.to(device)

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [9]:
state = ev.eval_iou(model,dataloader_val,device=device,num_classes=21,batch_size=batch_size)

In [14]:
print('iou:',state.metrics['mean IoU'],'Accuracy:',state.metrics['accuracy'],'Loss CE',state.metrics['CE Loss'])

iou: 0.6976822281981603 Accuracy: 0.9204657818788563 Loss CE 0.22761042032061754


In [10]:
! ls /data/model

SVM_transfer_30_voc.pt		     deeplabv3_unsupervised_voc.pt  fcn16_bn.pt
cnn_classif_transfer_30_voc.pt	     deeplabv3_voc.pt		    fcn32.pt
deeplabv3_10_autosup_voc.pt	     dlv3_voc_sbd_100.pt	    fcn32_bn.pt
deeplabv3_30_semisup_g10_l2_voc.pt   dlv3_voc_sbd_30.pt		    fcn8.pt
deeplabv3_30_semisup_g50_alt_voc.pt  dlv3_voc_sbd_rotate_30.pt	    fcn8_bn.pt
deeplabv3_30_semisup_voc.pt	     dlv3_voc_sbd_semisup_g0.pt     iou
deeplabv3_30_voc.pt		     dlv3_voc_sbd_semisup_g05.pt    loss
deeplabv3_50_voc.pt		     dlv3_voc_sbd_semisup_g1.pt
deeplabv3_autosupervised_voc07.pt    fcn16.pt
